<a name="top"></a>
# Using SQL in Python
Python needs a driver to access the RDBMS; and different RDBMS may require different driver. Here we will use [sqlite3](https://docs.python.org/3/library/sqlite3.html#module-sqlite3) since it is a self-contained, serverless,
zero-configuration, transactional SQL database
engine. It is perfect for beginners.

In this tutorial, we will cover,
1. [Connect to SQL Server](#connect_server)
2. [Create Table](#create_table)
3. [Manipulate Table](#manipulate_table)
4. [Delete Table](#delete_table)

<a name="connect_server"></a>
## 1. Connect to SQL Server

In [1]:
import sqlite3
conn = sqlite3.connect('example.db')
print(conn)

`sqlite3.connect` creates a new database if the database with the specified name doesn't exist; otherwise, it opens the database. We interact with SQL database through a Cursor object.

In [2]:
c = conn.cursor()
print(c)

[[top](#top)]

<a name="create_table"></a>
## 2. Create Table
A database contains table(s). To create a table, use the `CREATE TABLE` statement.

In [3]:
c.execute('''CREATE TABLE stocks
        (date date, 
        trans text, 
        symbol text, 
        qty real, 
        price real)''')

Here is the short list of SQL data types that we usually use.

| Data Type | Description |
|-----------|---------------------------|
| char(n)   | Fixed length string |
| text      | A string |
| integer   | Signed integer |
| real      | Floating precision number |
| date      | Date |

(For [more details](https://www.w3schools.com/sql/sql_datatypes.asp))

[[top](#top)]

<a name="manipulate_table"></a>
## 3. Manipulate Table
We can do 4 basic functions to records:
1. [Create](#create) ==> INSERT
2. [Read](#read) ==> SELECT
3. [Update](#update) ==> UPDATE
4. [Delete](#delete) ==> DELETE

<a name="create"></a>
### 3.1 Insert
Use Cursor object to execute the "INSERT INTO" statement. And use `conn.commit` to apply changes.

In [4]:
c.execute("INSERT INTO stocks VALUES ('2018-01-12','BUY','RHAT',100,35.14)")
conn.commit()

To insert multiple rows, use `executemany` function. 

In [5]:
purchases = [('2018-04-12', 'BUY', 'IBM', 1000, 45.00),
            ('2018-08-20', 'BUY', 'MSFT', 1000, 72.50),
            ('2018-12-25', 'SELL', 'IBM', 500, 53.00)]
c.executemany("INSERT INTO stocks VALUES (?,?,?,?,?)", purchases)

### SQL Injection Attack

Don’t assemble your query using Python’s string
operations because it is vulnerable to an SQL injection attack

**SQL injections** is a code injection technique, in which nefarious SQL statements are executed.

![SQL_injections](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

<a name="read"></a>
### 3.2 Select
Query is a request for data or information from a database table or combination of tables. Use `SELECT` statement to make a query from a table:

In [6]:
c.execute('SELECT * FROM stocks')
print(c.fetchone())

('2018-01-12', 'BUY', 'RHAT', 100.0, 35.14)


Use `fetchone` to retrieve one result record. Use `fetchall` to retrieve all result records.

In [7]:
c.execute('SELECT * FROM stocks')
for record in c.fetchall():
    print(record)

('2018-01-12', 'BUY', 'RHAT', 100.0, 35.14)
('2018-04-12', 'BUY', 'IBM', 1000.0, 45.0)
('2018-08-20', 'BUY', 'MSFT', 1000.0, 72.5)
('2018-12-25', 'SELL', 'IBM', 500.0, 53.0)


### Filter
To select only some columns, specify column name(s) after the `SELECT` statement

In [8]:
c.execute('SELECT trans, qty FROM stocks')
for record in c.fetchall():
    print(record)

('BUY', 100.0)
('BUY', 1000.0)
('BUY', 1000.0)
('SELL', 500.0)


To select only some rows, use `WHERE` statement to define the condition(s) of rows you want to include.

In [9]:
symbol = 'IBM'
c.execute("SELECT * FROM stocks WHERE symbol = '{}'".format(symbol))
for record in c.fetchall():
    print(record)

('2018-04-12', 'BUY', 'IBM', 1000.0, 45.0)
('2018-12-25', 'SELL', 'IBM', 500.0, 53.0)


### Primary Key
PRIMARY KEY is a column with a unique value for
each record.

In [10]:
c.execute('''CREATE TABLE brokers 
        (id integer primary key, 
        symbol text, 
        name text,
        age int)''')

`id` is a `integer primary key` column so it must contains an integer starting at 1, and increated by one for each record.

In [11]:
brokers = [('IBM', 'John', 40), 
          ('MSFT', 'Amy', 35), 
          ('RHAT', 'Matt', 55)]
c.executemany("INSERT INTO brokers(symbol, name, age) VALUES(?,?,?)",
             brokers)
conn.commit()

In [12]:
c.execute("SELECT * FROM brokers")
for record in c.fetchall():
    print(record)

(1, 'IBM', 'John', 40)
(2, 'MSFT', 'Amy', 35)
(3, 'RHAT', 'Matt', 55)


### Work with Multiple Tables
Relational databases always involve multiple tables. Use `JOIN` statment to combine rows from two or more tables. There are 4 ways to join. The most frequently used one is the `INNER JOIN`, which returns records that have matching values in both tables

![](https://www.w3schools.com/sql/img_innerjoin.gif)

In [13]:
c.execute('''SELECT * FROM stocks 
                INNER JOIN brokers 
                ON stocks.symbol = brokers.symbol''')
for record in c.fetchall():
    print(record)

('2018-01-12', 'BUY', 'RHAT', 100.0, 35.14, 3, 'RHAT', 'Matt', 55)
('2018-04-12', 'BUY', 'IBM', 1000.0, 45.0, 1, 'IBM', 'John', 40)
('2018-08-20', 'BUY', 'MSFT', 1000.0, 72.5, 2, 'MSFT', 'Amy', 35)
('2018-12-25', 'SELL', 'IBM', 500.0, 53.0, 1, 'IBM', 'John', 40)


Use `ON` the specify the matching conditions. For multiple conditions, use `AND` and `OR`.

#### Comparison operators in SQL

| Operator | Description |
|----------|-------------|
| =        | Equal to |
| >        | Greater than |
| <        | Less than |
| >=       | Greater than or equal to |
| <=       | Less than or equal to |
| <>       | Not equal to |

### Alias
Use `AS` to rename columns or tables.

In [14]:
c.execute('''SELECT * FROM stocks AS s
                INNER JOIN brokers AS b
                ON s.symbol = b.symbol''')
for record in c.fetchall():
    print(record)

('2018-01-12', 'BUY', 'RHAT', 100.0, 35.14, 3, 'RHAT', 'Matt', 55)
('2018-04-12', 'BUY', 'IBM', 1000.0, 45.0, 1, 'IBM', 'John', 40)
('2018-08-20', 'BUY', 'MSFT', 1000.0, 72.5, 2, 'MSFT', 'Amy', 35)
('2018-12-25', 'SELL', 'IBM', 500.0, 53.0, 1, 'IBM', 'John', 40)


[[top](#top)]

<a name="update"></a>
### 3.3 Update
To change values of existing records in a table, use `UPDATE...SET...WHERE` statement

In [15]:
symbol = 'RHAT'
c.execute('''UPDATE stocks SET qty = 200, price = 50
                WHERE symbol = "{}"'''.format(symbol))
c.execute('SELECT * FROM stocks')
for record in c.fetchall():
    print(record)

('2018-01-12', 'BUY', 'RHAT', 200.0, 50.0)
('2018-04-12', 'BUY', 'IBM', 1000.0, 45.0)
('2018-08-20', 'BUY', 'MSFT', 1000.0, 72.5)
('2018-12-25', 'SELL', 'IBM', 500.0, 53.0)


<a name="delete"></a>
### 3.4 Delete
To remove records, use `DELETE FROM` statement

In [16]:
symbol = 'RHAT'
c.execute('DELETE FROM stocks WHERE symbol = "{}"'.format(symbol))
conn.commit()
c.execute('SELECT * FROM stocks')
for record in c.fetchall():
    print(record)

('2018-04-12', 'BUY', 'IBM', 1000.0, 45.0)
('2018-08-20', 'BUY', 'MSFT', 1000.0, 72.5)
('2018-12-25', 'SELL', 'IBM', 500.0, 53.0)


Notice that we call `commit` after insertion or deletion, when changes in records occur.

[[top](#top)]

<a name="delete_table"></a>
## 4. Delete Table
To delete an existing table, use `DROP TABLE` statement

In [17]:
c.execute('DROP TABLE stocks')

Use the following code to get a list of tables.

In [18]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('brokers',)]


[[top](#top)]